In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import seaborn as sns
import xgboost as xgb
#from itertools import izip

In [3]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/unlabeled.csv')

sub_data =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [ ]:
#train_data.groupby('hospital_death').hist()
#list_plot = list(np.arange(169,173))

In [ ]:
# df = train_data.iloc[:,list_plot]
# df['hospital_death'] = train_data['hospital_death']

In [ ]:
#df.groupby('hospital_death').hist()
#train_data[['height','hospital_death']].groupby('hospital_death').hist()

In [4]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

In [5]:
train_data['weightclass'] = train_data['bmi'].map(weighted_classt)
test_data['weightclass'] = test_data['bmi'].map(weighted_classt)

In [6]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string','integer'])]['Variable Name'])
cat_var.remove('icu_admit_type')
cat_var.remove('bmi')
cat_var = cat_var[1:]

#to_label_encode = list(data_dict[data_dict['Data Type'].isin(['string'])]['Variable Name'])

In [7]:
cat_var = cat_var + ['weightclass']

In [8]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['numeric'])]['Variable Name'])
cont_var.remove('pred')
cont_var = cont_var + ['bmi']

#### Baseline

In [9]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [10]:
x_test = test_data.drop('hospital_death',axis=1)

In [11]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [12]:
x_comb.head()

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,weightclass
0,66154,25312,118,68.0,22.73,0,Caucasian,M,180.3,Floor,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,healthy weight
1,114252,59342,81,77.0,27.42,0,Caucasian,F,160.0,Floor,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,overweight
2,119783,50777,118,25.0,31.95,0,Caucasian,F,172.7,Emergency Department,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,class 1
3,79267,46918,118,81.0,22.64,1,Caucasian,F,165.1,Operating Room,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,healthy weight
4,92056,34377,33,19.0,NaN,0,Caucasian,M,188.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma,NaN


In [13]:
list_drop = ['encounter_id', 'patient_id', 'readmission_status']

In [14]:
# # Remove Features with more than 70 percent missing values
#x_comb = x_comb.drop(['readmission_status'],axis = 1)
x_comb = x_comb.drop(list_drop,axis = 1)

In [15]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [16]:
#Rajesh features

In [17]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))


100%|██████████| 30/30 [00:03<00:00,  8.03it/s]


In [18]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] =  x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [19]:
age_cat = []

for i in tqdm(range(0,len(x_comb))):
    
    val = x_comb['age'].iloc[i]
    
    if val >= 15 and val <= 24: 
        age_cat.append('igen')
    
    elif val >= 25 and val <= 54: 
       age_cat.append('Prime_working_Age')
    
    elif val >= 55 and val <= 64: 
        age_cat.append('Mature_working_Age')
        
    else: 
        age_cat.append('Elderly_working_Age')


100%|██████████| 131021/131021 [00:01<00:00, 75415.84it/s]


In [20]:
x_comb['age_category'] = age_cat
le = LabelEncoder()
x_comb['age_category'] = le.fit_transform(x_comb['age_category'].astype(str))

In [21]:
list_cols_nans = list(data_dict[data_dict['Category'].isin(['vitals','labs','labs blood gas'])]['Variable Name'])

In [22]:
list_cols_nans = list(set(list_cols_nans).intersection(set(x_comb.columns)))
#list_cols_nans

In [25]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [23]:
count_var = []
for i,col in tqdm(enumerate(new_cat_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])
    count_var.append(str(col)+'count')

30it [00:01, 18.18it/s]


In [26]:
for col in tqdm(new_cont_var):
    x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

100%|██████████| 153/153 [00:00<00:00, 558.54it/s]


In [27]:
#List to be biined -----> VITALS
to_binned_columns = list(data_dict[data_dict['Category'] == 'labs blood gas']['Variable Name'].unique())
to_binned_columns = list(set(to_binned_columns).intersection(set(x_comb.columns)))

In [28]:
binned_cols = []

for cols in tqdm(to_binned_columns) :
    x_comb[cols+'_binned'] = list(pd.cut(x_comb[cols], 10 ,labels=False))
    binned_cols.append(cols+'_binned')
    

100%|██████████| 16/16 [00:00<00:00, 29.40it/s]


In [29]:
x_comb['height'+'_binned'] = list(pd.cut(x_comb['height'], 8,labels=False))

In [30]:
featured_cols = binned_cols + ['age_category','height'+'_binned']

In [31]:
#2way interaction
bias_cols = ['gender','age']

In [32]:
inter_cols6 = []

for i in tqdm(bias_cols):
    
    for j in binned_cols :
        
        x_comb[i+'_'+j] = x_comb[i].astype(str) + '_' + x_comb[j].astype(str)
        
        inter_cols6.append(i+'_'+j)

100%|██████████| 2/2 [00:04<00:00,  2.41s/it]


In [33]:
#Critical -3way interaction


x_comb['age'+'gender'+'weighted_classt'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['weightclass'].astype(str)
x_comb['age'+'gender'+'ethnicity'] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str)
x_comb['age'+'gender'+'ethnicity' +'weighted_classt' ] = x_comb['age'].astype(str) + '_' + x_comb['gender'].astype(str) + '_' + x_comb['ethnicity'].astype(str) +'_' + x_comb['weightclass'].astype(str)


inter_cols6 = inter_cols6 + ['age'+'gender'+'weighted_classt'] +['age'+'gender'+'ethnicity'] + ['age'+'gender'+'ethnicity'+'weighted_classt']

In [34]:
for cols in tqdm(inter_cols6):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols].astype(str))

100%|██████████| 35/35 [00:01<00:00, 17.62it/s]


In [35]:
list_illness = ['aids',
                'cirrhosis',
                'diabetes_mellitus',
                'hepatic_failure',
                'immunosuppression',
                'leukemia',
                'lymphoma',
                'solid_tumor_with_metastasis']


In [36]:
inter_cols1 = []

for col in tqdm(list_illness) :
    x_comb['hospital_id'+str(col)] = x_comb['hospital_id'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['gender'+str(col)] = x_comb['gender'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['age'+str(col)] = x_comb['age'].astype(str) + '_' + x_comb[col].astype(str)
    x_comb['bmi'+str(col)] = x_comb['bmi'].astype(str) + '_' + x_comb[col].astype(str)
    
    inter_cols1.append('hospital_id'+str(col))
    inter_cols1.append('gender'+str(col))
    inter_cols1.append('age'+str(col))
    inter_cols1.append('bmi'+str(col))

100%|██████████| 8/8 [00:04<00:00,  1.69it/s]


In [37]:
for cols in tqdm(inter_cols1):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 32/32 [00:01<00:00, 24.32it/s]


In [38]:
inter_cols2 = []

for col in tqdm(list_illness) :
    x_comb['age_category'+str(col)] = x_comb['age_category'].astype(str) + '_' + x_comb[col].astype(str)
    inter_cols2.append('age_category'+str(col))

100%|██████████| 8/8 [00:01<00:00,  6.47it/s]


In [39]:
for cols in tqdm(inter_cols2):
    le = LabelEncoder()
    x_comb[cols] = le.fit_transform(x_comb[cols])
    

100%|██████████| 8/8 [00:00<00:00, 36.28it/s]


In [40]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']


In [41]:
cat_count_var = []

for i,col in tqdm(enumerate(new_cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m= 3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

30it [00:00, 31.57it/s]


In [42]:
# aggs = {}
# aggs['apache_4a_hospital_death_prob'] = ['median', 'mean', 'max', 'min']
# aggs['apache_4a_icu_death_prob'] = ['median', 'mean', 'max', 'min']


In [43]:
new_cat_var = list(set(new_cat_var).intersection(x_comb.columns))

In [44]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.9]

x_comb = x_comb.drop(columns = data_missing)

In [45]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))
#x_comb.isna().sum()

In [46]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

100%|██████████| 19/19 [00:04<00:00,  4.14it/s]


In [48]:
def mode(x):
    return Counter(x).most_common(1)[0][0]


In [49]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['nunique','count',mode]
    

100%|██████████| 30/30 [00:00<00:00, 162991.09it/s]


In [50]:
new_cat_var = list(set(new_cat_var).intersection(set(x_comb.columns)))

In [51]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    #aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])

100%|██████████| 30/30 [06:07<00:00, 12.26s/it]


In [52]:
x_comb['hospital_icu_id'] = x_comb['hospital_id'].astype(str) + '_' +  x_comb['icu_id'].astype(str)

le = LabelEncoder()
x_comb['hospital_icu_id'] = le.fit_transform(x_comb['hospital_icu_id'].astype(str))

In [53]:
#x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [55]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [56]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [65]:
xgb_model = xgb.XGBClassifier(
             silent = 1,
             n_estimators= 2000,
             show_progress = True,
             max_depth= 10,
             min_child_weight= 2,
             learning_rate=0.01,
             gamma=0.9,                        
             subsample=0.9,
             colsample_bytree=0.8,
             objective= 'binary:logistic',
             nthread= -1,
             scale_pos_weight=1)

In [66]:
xgb_model.fit(np.array(x1_train), np.array(y1_train))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.9,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=2000, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              show_progress=True, silent=1, subsample=0.9, verbosity=1)

In [67]:
pred = xgb_model.predict_proba(np.array(test_x))[:,1]

In [68]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = pred

/Users/s0c02nj/anaconda2/envs/graph/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [70]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/xgboost_20thfeb.csv',index = False)

In [ ]:
# random_state = 42
# np.random.seed(random_state)

# lgb_params = {
#     'boost': 'gbdt',
#     'learning_rate': 0.01,
#     'max_depth':6,
#     'metric':{'auc'},
#     'num_threads': -1,
#     'objective': 'binary',
#     'verbosity': 1,
#     'num_leaves':15,
#     'bagging_fraction':0.5, #rows
#     'bagging_frequency':5,
#     'min_data_in_leaf':50,
#     'feature_fraction':0.2, # features
#     'lambda_l1':10,
#     'lambda_l2':15,
#     'bagging_seed':1024,
#     'min_gain_split':0.3
# }

In [ ]:
# trn_data = lgb.Dataset(x1_train, label= y1_train)
# val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [ ]:
# lgb_clf = lgb.train(lgb_params,
#                     trn_data,
#                     13000,
#                     valid_sets = [trn_data, val_data],
#                     early_stopping_rounds=1500,
#                     verbose_eval=200
#                     )

In [ ]:
#predictions = lgb_clf.predict(test_x)

In [ ]:
# data_sub = x_test[['encounter_id']]
# data_sub['hospital_death'] = predictions

In [ ]:
#data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/single_model_besttillnow_19thfeb.csv',index = False)


In [ ]:
feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
list_unimp = list(feature_imp[feature_imp['Value'] == 0]['Feature'])

In [ ]:
# fig, ax = plt.subplots(figsize=(12,30))
# lgb.plot_importance(lgb_clf, max_num_features=130, height=0.8, ax=ax)
# ax.grid(False)
# plt.title("LightGBM - Feature Importance", fontsize=12)
# plt.show()
# #data_sub

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=35, shuffle=True)

In [ ]:
predictions = np.zeros(len(x_test))
num_models = 0

for train_index, valid_index in tqdm(kf.split(train_x, y_train)):
    
    d_train = lgb.Dataset(train_x.iloc[train_index], label=y_train[train_index])
    d_val = lgb.Dataset(train_x.iloc[valid_index], label=y_train[valid_index])

    clf = lgb.train(lgb_params, d_train, 20000, verbose_eval=1000, 
                    valid_sets = [d_train, d_val], early_stopping_rounds = 1500)
    

    
    num_models += 1
    
    predictions += clf.predict(test_x)

In [ ]:
predictions_10 = predictions /10.0
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = predictions_10

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/fold10_seed_rs_lgb_19thfeb.csv',index = False)


In [ ]:
#feature_imp = pd.DataFrame(sorted(zip(lgb_clf.feature_importance(), 
                                      x_comb.columns), reverse=True), columns=['Value','Feature'])

In [ ]:
#feature_imp[feature_imp['Feature'] == 'weightclass']


In [ ]:
#feature_imp[feature_imp['Value'] == 0]

In [ ]:
a1 =  pd.read_csv('/Users/s0c02nj/Desktop/sub_10fold_keep_missing_as_it_is_18thFeb2020_newfeat.csv')
a2 =  pd.read_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_12thfeb.csv')

In [ ]:
# a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4_lgbold_10thfeb_new_bmi_impute.csv')
# a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_10thFeb2020_lgb_type4_ff_20_na_bmi.csv')
# #a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_9thFeb2020_lgb_type3_ff_27.csv')
# a3 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_type1_type2_ensemble_5thFeb2020.csv')
# a4 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_11thFeb2020_lgb_type3_ff_27.csv')
# a5 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/pred_lgb_keeping_cat_cont_interchanged_11th_feb.csv')
# a6 = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/ensemble4tillnpw_11thfeb.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (a1['hospital_death'] + a2['hospital_death']
           )/2.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/OneDrive - Walmart Inc/WiDS/ensemble_till_now_18thfeb.csv',index = False)

In [ ]:
# def hospital_type(x):
    
#     if pd.isna(x):
#         return np.nan
    
#     elif x <= 4:
#         return "less_icu"
    
#     elif x > 8:
#         return "more_icu"
#     else:
#         return "med_icu"
